In [1]:
from webdriver_manager.chrome import ChromeDriverManager
from subprocess import CREATE_NO_WINDOW
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

from bs4 import BeautifulSoup as bs
import pandas as pd

import pyperclip
import time
import datetime
import os, sys


In [2]:
def find_css(css_selector):
    return browser.find_element(By.CSS_SELECTOR, css_selector)

In [3]:
def top_to_bottom(browser, i = None):
    start = time.time()
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        time.sleep(2.5)

        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            time.sleep(1)
            new_height = browser.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                return
        last_height = new_height  
        if i:
            if (time.time() - start) > i:
                break

In [4]:
def print_csv(f_name, name, contact, city):
    try:
        df = pd.DataFrame({"이름":name,
                            "연락처":contact,
                            "지역":city})
        df = df.fillna('null')
        numbers = df['연락처'].str.split('년').str.get(0).str[-4:]
        int_values = []
        for item in numbers:
            if item.isdigit():
                int_values.append(int(item))
            else:
                int_values.append('없음')
        df.isnull().sum()
        df = df.fillna('null')
        numbers = df['연락처'].str.split('년').str.get(0).str[-4:]
        int_values = []
        for item in numbers:
            if item.isdigit():
                int_values.append(int(item))
            else:
                int_values.append('없음')
        df['출생연도'] = int_values
        df.columns = ['이름', '연락처', '지역', '출생연도']
        df['지역'] = df['지역'].str.split('이전 거주지').str.get(1).str.split('현재').str.get(0).fillna('표시할 장소 없음')     
        
        if len(df[df['연락처'].str.contains('\+82')]) >= 1:
            df['연락처'] = df['연락처'].str.split('\+82 ').str.get(1).str[0:12].fillna('없음')
            df.index[df['연락처'].str.contains('1')]
            phone_num = df.index[df['연락처'].str.contains('1')]
            df.loc[phone_num, '연락처'] = "0" + df.loc[phone_num, '연락처']
        else:
            df['연락처'] = '없음'
        df.to_csv(f"{f_name}.csv", encoding='UTF-8')        
    except Exception as ex:
#         print(f'{len(name)} {len(contact)} {len(city)}')
        df = pd.DataFrame({"이름":name,
                            "연락처":contact,
                            "지역":city})
#         print(ex)
        df.to_csv(f"{f_name}.csv", encoding='UTF-8')

In [5]:
def open_browser():
    options = webdriver.ChromeOptions()
    options.add_argument("--proxy-server=socks5://127.0.0.1:9150")
    options.add_argument('window-size=1920x1080')
    options.add_argument('disable-gpu')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    options.add_argument('--no-sandbox')
    options.add_argument('no-sandox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--start-maximized')
    options.add_argument('incognito')

    # options.add_argument('headless')
    # Header Setting
    # service = Service(ChromeDriverManager().install())
    # service.creationflags = CREATE_NO_WINDOW
    # browser = webdriver.Chrome(options=options)
    chrome_service = Service('chromedriver')
    chrome_service.creationflags = CREATE_NO_WINDOW
    chrome_service = Service(executable_path="chromedriver.exe")
    browser = webdriver.Chrome(service=chrome_service, options=options)
    #open facebook
    try:
        browser.get("https://www.facebook.com/")
        time.sleep(2)
    except:
        return None
    
    return browser

In [6]:
def login(browser):
    #log info, search keyword
    USER = 'yms06034@gmail.com'
    PWD = 'mmk445566@'

    #login
    elem_id = browser.find_element("id", "email")
    pyperclip.copy(USER)
    elem_id.send_keys(Keys.CONTROL, "v")
    time.sleep(2)

    elem_pw = browser.find_element("id", "pass")
    pyperclip.copy(PWD)
    elem_pw.send_keys(Keys.CONTROL, "v")
    elem_pw.send_keys("\n")
    time.sleep(3)

    if "?sk=welcome" in browser.current_url:
        return 1
    else:
        return 0

In [7]:
def block_alert(browser):
    #except block alert
    try:
        elem_block = browser.find_element("xpath", "/html/body/div[3]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div[2]/div")
        elem_block.click()
    except:
        pass
    try:
        elem_block = browser.find_element('xpath', '/html/body/div[5]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div[2]/div')
        elem_block.click()
    except:
        pass
    time.sleep(1)

In [9]:
final_href = [
    'https://www.facebook.com/groups/181307315852800/members',
    'https://www.facebook.com/groups/915098498571012/members',
    'https://www.facebook.com/groups/332179336988536/members',
    'https://www.facebook.com/groups/2194827140563432/members'
]

#save elements
name, contact, city = [], [], []
error = []

In [10]:
browser = open_browser()
login(browser)

#loop with groups
for i in final_href:
    #open the group page
    browser.get(i)

    #except block alert
    block_alert(browser)

# scroll to bottom
    member = find_css('h2 > span > span > span > strong').text
    member = member.replace(',', '')
    member = 10
    while True:
        top_to_bottom(browser)
        time.sleep(1.5)

        #get pofile links
        soup_profile = bs(browser.page_source, "html.parser")
        h_list = soup_profile.find_all('h1')
        top_elements = soup_profile.find_all(class_="x1lq5wgf xgqcy7u x30kzoy x9jhf4c x1lliihq")

        #get profile links
        profile_list = []
        profile_href = []
        for j in top_elements:
            profile_list.append(j.find("a").attrs['href'])

        if len(profile_list) >= (int(member)*0.9):
            break

    #get profile lists
    for j in profile_list:
        profile_href.append('https://www.facebook.com/profile.php?id='+j[j.index('user/')+5:-1])

    #get group name
    for h in h_list:
        if h.text == '알림': continue
        elif h.text == '검색결과': continue
        else: g_name = h.text
    print(g_name)

    url = 'php?id='
    print(len(profile_href))

    #loop with profiles
    for j in profile_href:
        try:
            print(profile_href.index(j))
            #open the profile page
            browser.get(j)
            #is profile link with id or numbers
            temp = browser.current_url
            if url in temp:
                link = j + '&sk='
            else:
                link = temp[:temp.index('?')]
            #open 
            browser.get(link+"about_contact_and_basic_info")
            soup_info = bs(browser.page_source, "html.parser")
            soup_name = soup_info.find(class_="x78zum5 x15sbx0n x5oxk1f x1jxijyj xym1h4x xuy2c7u x1ltux0g xc9uqle")
            soup_info_contact = soup_info.find(class_="xyamay9 xqmdsaz x1gan7if x1swvt13")
            t_cont = soup_info_contact.text
            if '+82 ' in t_cont:
                contact.append(t_cont)
                name.append(soup_name.find("h1").text)
                browser.get(link+"about_places")
                soup_info = bs(browser.page_source, "html.parser")
                soup_info_city = soup_info.find(class_="xyamay9 xqmdsaz x1gan7if x1swvt13")
                city.append(soup_info_city.text)
            else: continue

        except:
            l_name = len(name)
            l_cont = len(contact)
            l_city = len(city)
            print(browser.current_url)
            error.append(browser.current_url)
            if l_name > l_cont:
                contact.append('이전 거주지표시할 장소 없음')
                try:
                    browser.get(link+"about_places")
                    soup_info = bs(browser.page_source, "html.parser")
                    soup_info_ct = soup_info.find(class_="xyamay9 xqmdsaz x1gan7if x1swvt13")
                    city.append(soup_info_ct.text)
                except:
                    city.append('')
            elif l_cont > l_city:
                city.append('')
            else: 
                continue

    # get real time
    timestamp = datetime.datetime.now()
    t_stamp = timestamp.strftime("%Y-%m-%d_%H%M%S")
    t_filename = f'_{t_stamp}'

    if len(name) == 0:
        print(f' nothing in there')
        continue
    else:
        print_csv(t_filename, name, contact, city)
        print(f' {t_filename}')


if len(name) == 0:
    print(f'nothing in there')
else:
    print_csv(t_filename, name, contact, city)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"h2 > span > span > span > strong"}
  (Session info: chrome=114.0.5735.199); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00E6A813+48355]
	(No symbol) [0x00DFC4B1]
	(No symbol) [0x00D05358]
	(No symbol) [0x00D309A5]
	(No symbol) [0x00D30B3B]
	(No symbol) [0x00D5E232]
	(No symbol) [0x00D4A784]
	(No symbol) [0x00D5C922]
	(No symbol) [0x00D4A536]
	(No symbol) [0x00D282DC]
	(No symbol) [0x00D293DD]
	GetHandleVerifier [0x010CAABD+2539405]
	GetHandleVerifier [0x0110A78F+2800735]
	GetHandleVerifier [0x0110456C+2775612]
	GetHandleVerifier [0x00EF51E0+616112]
	(No symbol) [0x00E05F8C]
	(No symbol) [0x00E02328]
	(No symbol) [0x00E0240B]
	(No symbol) [0x00DF4FF7]
	BaseThreadInitThunk [0x75A57D59+25]
	RtlInitializeExceptionChain [0x7740B79B+107]

In [12]:
member.text

'  · 5,122'